In [1]:
import sys
import os

In [5]:
import nltk
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/astridz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/astridz/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

https://blog.epsilla.com/build-a-chatbot-that-runs-purely-on-your-local-machine-using-llama-2-epsilla-langchain-a5053b16f85

# Helper Function

In [2]:
def reading(fileName):
    f = open(fileName, "r")
    content = f.read() 
    f.close()
    return content

In [3]:
def writing(input, filename):
    f = open(filename, 'w')
    f.write(input)
    f.close()

# Load web Page

In [4]:
from langchain.document_loaders import WebBaseLoader
web = "https://www.mtholyoke.edu/academics/find-your-program/computer-science"
loader = WebBaseLoader(web)

In [5]:
cs_data = loader.load()

# Install Epsilla Vector Database

Run these 2 commands to install the Epsilla vector database docker image on your personal computer:

https://github.com/epsilla-cloud/vectordb

In [6]:
import os
# docker installed in "/Users/astridz/Documents/Moho_Bot/myvenv/lib/python3.10/site-packages/docker"
!docker pull epsilla/vectordb
!docker run --pull=always -d -p 8888:8888 epsilla/vectordb

Using default tag: latest
latest: Pulling from epsilla/vectordb
Digest: sha256:c4fbafa4a0055c3aa5750db7602ce507f894184d4fb769559296412b2a48f7b3
Status: Image is up to date for epsilla/vectordb:latest
docker.io/epsilla/vectordb:latest

What's Next?
  View a summary of image vulnerabilities and recommendations → docker scout quickview epsilla/vectordb
latest: Pulling from epsilla/vectordb
Digest: sha256:c4fbafa4a0055c3aa5750db7602ce507f894184d4fb769559296412b2a48f7b3
Status: Image is up to date for epsilla/vectordb:latest
c8574050567ae5045672fa4f985b1d44372a231067c35055f0c4af066ba46fb8
docker: Error response from daemon: driver failed programming external connectivity on endpoint naughty_ishizaka (c27776ba064260427bdc0fc7245bbdd16b2da1ab0c03239418c32f5b7f1fc8a8): Bind for 0.0.0.0:8888 failed: port is already allocated.


## Preprocessing the data

**Tokenize** the data_string:

In [7]:
import nltk
import ssl
from nltk.tokenize import word_tokenize, sent_tokenize
from transformers import AutoTokenizer, AutoModel

/Users/astridz/Documents/Moho_Bot/myvenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/astridz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
from operator import itemgetter
import string
from nltk.stem.snowball import PorterStemmer
from nltk.corpus import stopwords
from collections import Counter
from scipy.interpolate import make_interp_spline
import numpy as np
import pandas as pd
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/astridz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
def preprocessing(data, stopping, stemming):# Tokenize each document in the list
    tokenized_documents = []
    trans = str.maketrans('','', string.punctuation)
    sentences = nltk.sent_tokenize(data)  # Tokenize the document into sentences
    for each in sentences:
            #Tokenization: remove punctuation
            each = each.translate(trans)
            #content clear
            each = each.replace ("\n", " ")
            #Case folding: make lowercase
            each = each.lower()
            words = [nltk.word_tokenize(each)]
            # Tokenize each sentence into words
            tokenized_documents.extend(words)

    stemmer = PorterStemmer()
    stoplist = set(stopwords.words('english'))
 
    eachline = []
    output = []
    
    for sentences in tokenized_documents:
            #Stopping & Normalization option
            if (stopping == True):
                filtered_stop = [word for word in sentences if word not in stoplist]
                if (stemming == True):
                    stem_words = filtered_stop
                    stem_words  = [stemmer.stem(w) for w in stem_words]
                    eachline= ' '.join(stem_words)
                else:
                    eachline = ' '.join(filtered_stop)
            else:
                if (stemming == True):
                    stem_words = sentences
                    stem_words  = [stemmer.stem(w) for w in stem_words]
                    eachline= ' '.join(stem_words)
                else:
                    eachline= each
            eachline = [eachline]
            output.extend(eachline)
    # print(output)
    output_string = " ".join(output)
    return  output_string

In [11]:
# store tokenizing data into ".txt" file
def create_db(data, filename):
    if (os.getcwd() != "/Users/astridz/Documents/Moho_Bot/Documents"):
        os.chdir('Documents')
        
    data_string = data[0].page_content
    data_string = data_string.replace("\n", "")
    clean_data = preprocessing(data_string, True, False)
    
    writing(clean_data, filename)
    os.chdir('../')
    AssertionError(os.getcwd() != "/Users/astridz/Documents")

Get data from web page

In [12]:
filename = "cs_page.txt"
create_db(cs_data, filename)

**Embedding**: split the document, embed the chunks, and store them into Epsilla

In [13]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Epsilla
from pyepsilla import vectordb
from sentence_transformers import SentenceTransformer

from typing import List
from glob import glob

In [14]:
# Local embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [15]:
class LocalEmbeddings():
  def embed_documents(self, texts: List[str]) -> List[List[float]]:
    return model.encode(texts).tolist()
  
embeddings = LocalEmbeddings()

In [16]:
# Get list of all files in "./documents/"
files = glob("./documents/*")

for file in files:
  loader = TextLoader(file)
  documents = loader.load()
  splitted_documents = CharacterTextSplitter(
    separator='\n', chunk_size=1000, 
    chunk_overlap=200).split_documents(documents)

  client = vectordb.Client()
  
  # Connect to Epsilla as knowledge base.
  vector_store = Epsilla.from_documents(
    splitted_documents,
    embeddings,
    client,
    db_path="/tmp/localchatdb",
    db_name="LocalChatDB",
    collection_name="LocalChatCollection"
  )

[INFO] Connected to localhost:8888 successfully.
[INFO] Connected to localhost:8888 successfully.


# Import Llama with llama.cpp

In [17]:
# Download the model file
model_name = 'TheBloke/Llama-2-7b-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf'
pure_name = model_name.split('/')[-1]
parts = model_name.split('/')
model_path = f"{parts[0]}/{parts[1]}"
print("Your model name is ", pure_name)

Your model name is  llama-2-7b-chat.Q4_K_M.gguf


In [18]:
os.chdir('/Users/astridz/Documents')
if not os.path.exists('llama.cpp'):
    print("Cloning llama.cpp...")
    !git clone https://github.com/ggerganov/llama.cpp
    %cd llama.cpp
    print("Compiling for Mac with M1 chip...")
    !LLAMA_METAL=1 make
    print("Compilation completed!")
    %cd ../
    %cd Moho.Bot
else:
    print("llama.cpp has already been cloned into this directory!")
    %cd MOHO_BOT
print("current directory is ", os.getcwd())

llama.cpp has already been cloned into this directory!
/Users/astridz/Documents/Moho_Bot
current directory is  /Users/astridz/Documents/Moho_Bot


In [19]:
# set directory to llama.cpp
os.chdir('/Users/astridz/Documents/llama.cpp/')
if not os.path.exists(pure_name):
    !wget https://huggingface.co/{model_name}
else:
    print(f"{pure_name} already exists!")
%cd ../Moho_bot

llama-2-7b-chat.Q4_K_M.gguf already exists!
/Users/astridz/Documents/Moho_Bot


# RUN

In [3]:
import streamlit as st
import subprocess
import os

Initialize streamlit

In [21]:
# Check if 'key' already exists in session_state
# If not, then initialize it
if 'key' not in st.session_state:
    st.session_state['key'] = 'value'

# Session State also supports the attribute based syntax
if 'key' not in st.session_state:
    st.session_state.key = 'value'

2023-10-24 16:11:37.696 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


In [19]:
# Testing: Connect to Llama
model_name = 'TheBloke/Llama-2-7b-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf'
pure_name = model_name.split('/')[-1]
parts = model_name.split('/')
model_path = f"{parts[0]}/{parts[1]}"

prompt = f'''
    Answer the Question based on the given Context. Try to understand the Context and rephrase them.
    Please don't make things up or say things not mentioned in the Context. Ask for more information when needed.
    '''
    
# WaitReview: call model
os.chdir('/Users/astridz/Documents/llama.cpp')
# command = ./main -m llama-2-7b-chat.Q4_K_M.gguf  -n 1024 -ngl 48
# command = ['./main', '-m', pure_name, '-ngl 48', prompt]
# ./main -m ./models/7B/ggml-model-q4_0.bin -n 1024 --repeat_penalty 1.0 --color -i -r "User:" -f ./prompts/chat-with-bob.txt
command = f"./main -m {pure_name} -c 1024 -ngl 48 {prompt}"
f"./main -m {pure_name} -c 1024 -ngl 48 {prompt}"
print(command)
process = subprocess.Popen(command, shell=True,stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
# out, err = process.communicate()

# if err:
#     print("error occurred: ",err.decode())
output = process.stdout.read()
content = ''
if output:
    content = content + output
return_code = process.poll()

# Change back to the parent directory using Python
os.chdir('../')

./main -m llama-2-7b-chat.Q4_K_M.gguf -c 1024 -ngl 48 
    Answer the Question based on the given Context. Try to understand the Context and rephrase them.
    Please don't make things up or say things not mentioned in the Context. Ask for more information when needed.
    


Create a QA Chain using Llama2: